In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import time


In [3]:
def get_g_stats(soup):
    teams = soup.find('h1').text.split(' @ ')
    home = teams[1]
    away = teams[0]
    date = soup.find('h2').text.split('\n')[0]

    g = soup.find_all('table', {'id': re.compile(r'tb[A-Z]{2,3}stgall')})

    gs = []


    df = pd.read_html(str(g[0]))[0]
    df['Team'] = home
    df['Opposition'] = away
    df['Venue'] = 'home'
    gs.append(df)
    df = pd.read_html(str(g[5]))[0]
    df['Team'] = away
    df['Opposition'] = home
    df['Venue'] = 'away'
    gs.append(df)
    gs_master = pd.concat(gs)
    gs_master['Date'] = date
    gs_master = gs_master[['Date', 'Player', 'Venue', 'Team', 'Opposition', 'TOI', 'Goals Against', 'Expected Goals Against']]
    return gs_master
    

In [4]:
def get_rel_stats(soup):
    teams = soup.find('h1').text.split(' @ ')
    home = teams[1]
    away = teams[0]
    date = soup.find('h2').text.split('\n')[0]

    ind = soup.find_all('table', {'id': re.compile(r'tb[A-Z]{2,3}stall')})
    oi = soup.find_all('table', {'id': re.compile(r'tb[A-Z]{2,3}oi5v5')})
    ois = []
    inds = []

    df = pd.read_html(str(oi[0]))[0]
    df['Team'] = home
    df['Opposition'] = away
    df['Venue'] = 'home'

    ois.append(df)
    df = pd.read_html(str(oi[1]))[0]
    df['Team'] = away
    df['Opposition'] = home
    df['Venue'] = 'away'

    ois.append(df)

    df = pd.read_html(str(ind[0]))[0]
    df['Team'] = home
    df['Opposition'] = away
    df['Venue'] = 'home'

    inds.append(df)
    df = pd.read_html(str(ind[1]))[0]

    df['Team'] = away
    df['Opposition'] = home
    df['Venue'] = 'away'

    inds.append(df)

    oi_master = pd.concat(ois)
    oi_master['Date'] = date
    oi_master = oi_master[['Date', 'Player', 'Venue', 'Team', 'Opposition', 'TOI', 'GF', 'GA', 'xGF', 'xGA']]
    ind_master = pd.concat(inds)
    ind_master['Date'] = date
    ind_master = ind_master[['Date', 'Player', 'Venue', 'Team', 'Opposition', 'Position', 'TOI', 'Goals', 'First Assists', 'Second Assists', 'ixG', 'Shots Blocked', 'Total Penalties', 'Penalties Drawn', 'Faceoffs Won', 'Faceoffs Lost']]

    df = ind_master.merge(oi_master, on=['Player', 'Team'])
    return df.drop(columns = ['Date_y', 'Venue_y', 'TOI_y', 'Opposition_y'])

In [8]:
from datetime import datetime
for year in [2022,]:
    print(year, "started at ", datetime.now())
    goalie_dfs = []
    dfs = []
    gameid = 20001
    url = f'https://www.naturalstattrick.com/game.php?season={year}{year+1}&game={gameid}'
    req = requests.get(url)
    soup = BeautifulSoup(req.content)
    dfs.append(get_rel_stats(soup))
    goalie_dfs.append(get_g_stats(soup))
    time.sleep(30)
    timer = 0
    errors = []
    while len(soup.text) > 3000:
        try:
            gameid += 1
            if timer == 100:
                time.sleep(661)
                timer = 0
            url = f'https://www.naturalstattrick.com/game.php?season={year}{year+1}&game={gameid}'
            req = requests.get(url)
            soup = BeautifulSoup(req.content)
            stats = get_rel_stats(soup)
            dfs.append(get_rel_stats(soup))
            goalie_dfs.append(get_g_stats(soup))
            time.sleep(30)
            timer += 1
        except:
            errors.append(gameid)
    pd.concat(dfs).to_csv(f'{year}{year+1}_skaters.csv', index=False)
    pd.concat(goalie_dfs).to_csv(f'{year}{year+1}_goalies.csv', index=False)
    print(year, errors)

2022 started at  2023-03-12 13:06:17.182190
2022 [20320, 20547]


In [9]:
from datetime import datetime
for year in [2022,]:
    print(year, "started at ", datetime.now())
    gameid = 20548
    url = f'https://www.naturalstattrick.com/game.php?season={year}{year+1}&game={gameid}'
    req = requests.get(url)
    soup = BeautifulSoup(req.content)
    dfs.append(get_rel_stats(soup))
    goalie_dfs.append(get_g_stats(soup))
    time.sleep(30)
    timer = 0
    errors = []
    while len(soup.text) > 3000:
        try:
            gameid += 1
            if timer == 100:
                time.sleep(661)
                timer = 0
            url = f'https://www.naturalstattrick.com/game.php?season={year}{year+1}&game={gameid}'
            req = requests.get(url)
            soup = BeautifulSoup(req.content)
            stats = get_rel_stats(soup)
            dfs.append(get_rel_stats(soup))
            goalie_dfs.append(get_g_stats(soup))
            time.sleep(30)
            timer += 1
        except:
            errors.append(gameid)
    pd.concat(dfs).to_csv(f'{year}{year+1}_skaters.csv', index=False)
    pd.concat(goalie_dfs).to_csv(f'{year}{year+1}_goalies.csv', index=False)
    print(year, errors)

2022 started at  2023-03-12 20:19:18.393405
2022 [21061]
